<a href="https://colab.research.google.com/github/Ameer-sayyad/S-NLP-PROJECTS/blob/main/OPUS_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade datasets

In [1]:
from datasets import load_dataset


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"
raw_datasets = load_dataset("cfilt/iitb-english-hindi")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "hi"
def preprocess_function(examples):
  inputs = [ex[source_lang] for ex in examples["translation"]]
  targets = [ex[target_lang] for ex in examples["translation"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf",pad_to_multiple_of=128)
small_train_dataset = tokenized_datasets["train"].select(range(1000))  # Use only first 1000 examples
train_dataset = model.prepare_tf_dataset(
    small_train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator
)

validation_dataset = model.prepare_tf_dataset(
  tokenized_datasets["validation"],
  batch_size=batch_size,
  shuffle=False,
  collate_fn=data_collator,
)
generation_dataset = model.prepare_tf_dataset(
  tokenized_datasets["validation"],
  batch_size=8,
  shuffle=False,
  collate_fn=generation_data_collator,
)
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)
model.save_pretrained("tf_model/")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

input_texts = ["I am good at English.", "This is my first NLP project."]

tokenized = tokenizer(input_texts, return_tensors='np', padding=True, truncation=True)
out = model.generate(**tokenized, max_length=128)

for i, output in enumerate(out):
    print(f"English: {input_texts[i]}")
    print(f"Hindi: {tokenizer.decode(output, skip_special_tokens=True)}\n")


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


English: I am good at English.
Hindi: मैं अंग्रेजी में अच्छा हूँ.

English: This is my first NLP project.
Hindi: यह मेरी पहली NLP परियोजना है.

